# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df=pd.read_csv('../output_data/cities.csv')
vacation_df.count()

City          546
Lat           546
Lng           546
Max Temp      546
Humidity      546
Cloudiness    546
Wind Speed    546
Country       543
Date          546
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

humidity= vacation_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
better_vacation_df= vacation_df.loc[(vacation_df["Max Temp"]>70 ) & (vacation_df["Max Temp"]< 80) & (vacation_df["Wind Speed"]<= 10) 
                                    & (vacation_df["Cloudiness"]==0)].dropna()
better_vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,choix,26.7092,-108.3219,71.06,55,0,4.34,MX,1634911116
43,cabo san lucas,22.8909,-109.9124,77.50,86,0,8.46,MX,1634911056
55,bikramganj,25.2000,84.2500,72.81,67,0,2.95,IN,1634911133
57,sananduva,-27.9497,-51.8067,70.47,58,0,4.65,BR,1634911134
137,tucuman,-26.8241,-65.2226,78.80,36,0,6.91,AR,1634911163
144,gohpur,26.8833,93.6333,79.05,87,0,3.60,IN,1634911166
145,cidreira,-30.1811,-50.2056,71.49,61,0,4.25,BR,1634911167
162,chail,25.4333,81.6333,77.54,51,0,5.82,IN,1634911173
199,birecik,37.0294,37.9903,78.01,19,0,1.12,TR,1634911187
201,areosa,41.7178,-8.8571,70.86,58,0,9.10,PT,1634911188


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:

hotel_df=better_vacation_df
hotel_df["Hotel Name"]= ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,choix,26.7092,-108.3219,71.06,55,0,4.34,MX,1634911116,
43,cabo san lucas,22.8909,-109.9124,77.50,86,0,8.46,MX,1634911056,
55,bikramganj,25.2000,84.2500,72.81,67,0,2.95,IN,1634911133,
57,sananduva,-27.9497,-51.8067,70.47,58,0,4.65,BR,1634911134,
137,tucuman,-26.8241,-65.2226,78.80,36,0,6.91,AR,1634911163,
144,gohpur,26.8833,93.6333,79.05,87,0,3.60,IN,1634911166,
145,cidreira,-30.1811,-50.2056,71.49,61,0,4.25,BR,1634911167,
162,chail,25.4333,81.6333,77.54,51,0,5.82,IN,1634911173,
199,birecik,37.0294,37.9903,78.01,19,0,1.12,TR,1634911187,
201,areosa,41.7178,-8.8571,70.86,58,0,9.10,PT,1634911188,


In [32]:
import pprint
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
       
            # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

            # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"

        
#     try:
        # make request and print url
        response = requests.get(base_url, params=params).json()
#         pprint.pprint(response)
#         break

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
#     try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,choix,26.7092,-108.3219,71.06,55,0,4.34,MX,1634911116,Hotel Montesclaros
43,cabo san lucas,22.8909,-109.9124,77.50,86,0,8.46,MX,1634911056,Hotel Tesoro Los Cabos
55,bikramganj,25.2000,84.2500,72.81,67,0,2.95,IN,1634911133,Hotel Jairam
57,sananduva,-27.9497,-51.8067,70.47,58,0,4.65,BR,1634911134,São José Palace Hotel
137,tucuman,-26.8241,-65.2226,78.80,36,0,6.91,AR,1634911163,Dallas Hotel Tucuman
144,gohpur,26.8833,93.6333,79.05,87,0,3.60,IN,1634911166,Murchana Music
145,cidreira,-30.1811,-50.2056,71.49,61,0,4.25,BR,1634911167,Hotel Castelo
162,chail,25.4333,81.6333,77.54,51,0,5.82,IN,1634911173,Chaudhary house
199,birecik,37.0294,37.9903,78.01,19,0,1.12,TR,1634911187,Birecik Ogretmenevi
201,areosa,41.7178,-8.8571,70.86,58,0,9.10,PT,1634911188,Hotel Flôr de Sal


In [33]:
# for index, row in hotel_df.iterrows():
#     print(row)
#     break


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# hotel_info

In [35]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content= hotel_info)

# Display figure
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))